# Science User Case - Inspecting a Candidate List

Ogle et al. (2016) mined the NASA/IPAC Extragalactic Database (NED) to identify a new type of galaxy: Superluminous Spiral Galaxies. Here's the paper: 

Here's the paper: https://ui.adsabs.harvard.edu//#abs/2016ApJ...817..109O/abstract

Table 1 lists the positions of these Super Spirals. Based on those positions, let's create multiwavelength cutouts for each super spiral to see what is unique about this new class of objects.

Import some python modules so we don't get inundated with unimportant warnings when we run our notebook cells.

In [184]:
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append('workshop-dev-master')
from navo_utils.image import Image, ImageColumn
from navo_utils.spectra import Spectra, SpectraColumn
from navo_utils.cone import Cone
from navo_utils.tap import Tap
from navo_utils.utils import astropy_table_from_votable_response
from astropy.table import Table

# Utility functions used in the workshop.
import navo_utils.utils

## Search NED for objects in this paper.

In [1]:
from astroquery.ned import Ned
objects_in_paper = Ned.query_refcode('2016ApJ...817..109O')
objects_in_paper.show_in_notebook()

idx,No.,Object Name,RA(deg),DEC(deg),Type,Velocity,Redshift,Redshift Flag,Magnitude and Filter,Distance (arcmin),References,Notes,Photometry Points,Positions,Redshift Points,Diameter Points,Associations
,,,degrees,degrees,,km / s,,,,arcm,,,,,,,
0,1,2MASX J00155012-1002427,3.95892,-10.045109999999999,G,52788.0,0.17608299999999999,,17.5g,--,10,0,51,5,3,10,0
1,2,2MASX J00380781-0109365,9.5325199999999999,-1.1602399999999999,G,62565.0,0.20869499999999999,,18.0g,--,10,0,59,7,3,10,0
2,3,2MASS J04042287-0541349,61.095480000000002,-5.6930300000000003,G,75138.0,0.250635,,18.6g,--,4,0,49,7,4,8,0
3,4,2MASX J07380615+2823592,114.52567999999999,28.39995,G,69255.0,0.23100999999999999,,18.0g,--,8,0,54,5,3,10,0
4,5,2MASX J07550424+1353261,118.76776,13.890650000000001,G,66754.0,0.22266900000000001,,18.5g,--,7,0,38,5,3,6,0
5,6,2MASX J08265512+1811476,126.72969000000001,18.196650000000002,G,79608.0,0.26554299999999997,,18.4g,--,4,0,34,4,3,6,0
6,7,2MASX J08542169+0449308,133.59032999999999,4.8251099999999996,G,47018.0,0.156835,,16.7g,--,12,0,58,6,3,10,0
7,8,GMBCG J137.43670+22.43538,137.4367,22.435390000000002,GClstr,90837.0,0.30299999999999999,PHOT,,--,2,0,0,0,1,0,0
8,9,2MASX J09094480+2226078,137.4367,22.435379999999999,G,85557.0,0.28538599999999997,,19.0g,--,5,0,30,3,4,6,0


## The NED query returns all objects in the paper, not just the galaxies in Table 1. Let's filter these results to only include the galaxies.

Hint: There is a difference between strings and byte strings

In [3]:
#Let's see whether we are dealing with strings or byte strings
objects_in_paper['Type'].data[0]

b'G'

In [4]:
#Byte strings it is!
galaxies = objects_in_paper[objects_in_paper['Type'] == b'G']

galaxies.show_in_notebook()

idx,No.,Object Name,RA(deg),DEC(deg),Type,Velocity,Redshift,Redshift Flag,Magnitude and Filter,Distance (arcmin),References,Notes,Photometry Points,Positions,Redshift Points,Diameter Points,Associations
,,,degrees,degrees,,km / s,,,,arcm,,,,,,,
0,1,2MASX J00155012-1002427,3.95892,-10.045109999999999,G,52788.0,0.17608299999999999,,17.5g,--,10,0,51,5,3,10,0
1,2,2MASX J00380781-0109365,9.5325199999999999,-1.1602399999999999,G,62565.0,0.20869499999999999,,18.0g,--,10,0,59,7,3,10,0
2,3,2MASS J04042287-0541349,61.095480000000002,-5.6930300000000003,G,75138.0,0.250635,,18.6g,--,4,0,49,7,4,8,0
3,4,2MASX J07380615+2823592,114.52567999999999,28.39995,G,69255.0,0.23100999999999999,,18.0g,--,8,0,54,5,3,10,0
4,5,2MASX J07550424+1353261,118.76776,13.890650000000001,G,66754.0,0.22266900000000001,,18.5g,--,7,0,38,5,3,6,0
5,6,2MASX J08265512+1811476,126.72969000000001,18.196650000000002,G,79608.0,0.26554299999999997,,18.4g,--,4,0,34,4,3,6,0
6,7,2MASX J08542169+0449308,133.59032999999999,4.8251099999999996,G,47018.0,0.156835,,16.7g,--,12,0,58,6,3,10,0
7,9,2MASX J09094480+2226078,137.4367,22.435379999999999,G,85557.0,0.28538599999999997,,19.0g,--,5,0,30,3,4,6,0
8,10,2MASX J09260805+2405242,141.53357,24.090039999999998,G,66689.0,0.22245100000000001,,17.8g,--,10,0,32,4,5,6,0


# Search the NAVO Registry for WISE, GALEX, and SDSS image services

In [6]:
from navo_utils.registry import Registry

wise_services = Registry.query(keyword='allwise', service_type='image')

print(f'{len(wise_services)} result(s) found.')

wise_services

1 result(s) found.


waveband,short_name,ivoid,res_description,access_url,reference_url,publisher,service_type
str8,str11,str39,str1031,str60,str58,str34,str17
infrared,AllWISE L3a,ivo://irsa.ipac/wise/images/allwise/l3a,"The AllWISE program builds upon the work of the successful Wide-field Infrared Survey Explorer mission (WISE; Wright et al. 2010) by combining data from the WISE cryogenic and NEOWISE (Mainzer et al. 2011 ApJ, 731, 53) post-cryogenic survey phases to form the most comprehensive view of the full mid-infrared sky currently available. By combining the data from two complete sky coverage epochs using an advanced data processing system, AllWISE has generated new products that have enhanced photometric sensitivity and accuracy, and improved astrometric precision compared to the 2012 WISE All-Sky Data Release. Exploiting the 6 to 12 month baseline between the WISE sky coverage epochs enables AllWISE to measure source motions for the first time, and to compute improved flux variability statistics. The AllWISE Images Atlas is comprised of 18,240 4-band calibrated 1.56°x1.56° FITS images, depth-of-coverage and noise maps, and image metadata produced by coadding nearly 7.9 million Single-exposure images from all survey phases.",https://irsa.ipac.caltech.edu/ibe/sia/wise/allwise/p3am_cdd?,http://wise2.ipac.caltech.edu/docs/release/allwise/expsup/,NASA/IPAC Infrared Science Archive,simpleimageaccess


In [7]:
galex_services = Registry.query(keyword='galex', service_type='image')

print(f'{len(galex_services)} result(s) found.')

galex_services

7 result(s) found.


waveband,short_name,ivoid,res_description,access_url,reference_url,publisher,service_type
str19,str11,str36,str1024,str153,str48,str41,str17
uv,GalexNear,ivo://nasa.heasarc/skyview/galexnear,"The GALEX, Galaxy Explorer, mission was launched by a Pegasus-XL vehicle on April 28 2003 into a 690km altitude, 29 degree inclination, circular orbit with a 98.6 minute period. The GALEX instrument allows imaging and spectroscopic observations to be made in two ultraviolet bands, Far UV (FUV) 1350-1780A and Near UV (NUV) 1770-2730A. The instrument provides simultaneous co-aligned FUV and NUV images with spatial resolution 4.3 and 5.3 arcseconds respectively. Details of the performance of the instrument and detectors can be found in Morrissey et al. (2007) ApJS, 173, 682. &amp;lt;p&amp;gt; The &amp;lt;i&amp;gt;SkyView&amp;lt;/i&amp;gt; GALEX surveys mosaic the intensity images of All-Sky Survey images. For a given pixel only the nearest image is used. Since a given GALEX observation is circular, this maximizes the coverage compared with default image finding algorithms which use the distance from edge of the image. &amp;lt;p&amp;gt; As of February 10, 2011, SkyView uses the GALEX GR6 data release.",https://skyview.gsfc.nasa.gov/cgi-bin/vo/sia.pl?survey=galexnear&amp;,https://skyview.gsfc.nasa.gov,NASA/GSFC HEASARC,simpleimageaccess
infrared#optical#uv,LVL,ivo://irsa.ipac/spitzer/images/lvl,"LVL consists of a sample of 258 galaxies, which have been mapped with both IRAC (4 bands) and MIPS (3 bands). In addition, ancillary data products consisting of images in the narrow-band H-alpha line emission and broad-band R (from the ground) and the UV continuum (2 bands) from GALEX are also available for many of the galaxies.",https://irsa.ipac.caltech.edu/cgi-bin/Atlas/nph-atlas?mission=LVL&amp;hdr_location=%5CLVLDataPath%5C&amp;SIAP_ACTIVE=1&amp;collection_desc=LVL&amp;,https://irsa.ipac.caltech.edu/data/SPITZER/LVL,NASA/IPAC Infrared Science Archive,simpleimageaccess
infrared,GOALS,ivo://irsa.ipac/spitzer/images/goals,"The Great Observatories All-sky LIRG Survey (GOALS) combines imaging and spectroscopic data from NASA’s Spitzer, Hubble, Chandra and GALEX observatories in a comprehensive survey of over 200 of the most luminous infrared-selected galaxies in the local Universe.",https://irsa.ipac.caltech.edu/cgi-bin/Atlas/nph-atlas?mission=GOALS&amp;hdr_location=%5CGOALSDataPath%5C&amp;SIAP_ACTIVE=1&amp;collection_desc=GOALS&amp;,https://irsa.ipac.caltech.edu/data/SPITZER/GOALS,NASA/IPAC Infrared Science Archive,simpleimageaccess
uv,GALEX,ivo://nasa.heasarc/skyview/galex,"The GALEX, Galaxy Explorer, mission was launched by a Pegasus-XL vehicle on April 28 2003 into a 690km altitude, 29 degree inclination, circular orbit with a 98.6 minute period. The GALEX instrument allows imaging and spectroscopic observations to be made in two ultraviolet bands, Far UV (FUV) 1350-1780A and Near UV (NUV) 1770-2730A. The instrument provides simultaneous co-aligned FUV and NUV images with spatial resolution 4.3 and 5.3 arcseconds respectively. Details of the performance of the instrument and detectors can be found in Morrissey et al. (2007) ApJS, 173, 682. &amp;lt;p&amp;gt; The &amp;lt;i&amp;gt;SkyView&amp;lt;/i&amp;gt; GALEX surveys mosaic the intensity images of All-Sky Survey images. For a given pixel only the nearest image is used. Since a given GALEX observation is circular, this maximizes the coverage compared with default image finding algorithms which use the distance from edge of the image. &amp;lt;p&amp;gt; As of February 10, 2011, SkyView uses the GALEX GR6 data release.",https://skyview.gsfc.nasa.gov/cgi-bin/vo/sia.pl?survey=galex&amp;,https://skyview.gsfc.nasa.gov,NASA/GSFC HEASARC,simpleimageaccess
uv,GalexFar,ivo://nasa.heasarc/skyview/galexfar,"The GALEX, Galaxy Explorer, mission was launched by a Pegasus-XL vehicle on April 28 2003 into a 690km altitude, 29 degree inclination, circular orbit with a 98.6 minute period. The GALEX instrument allows im

In [8]:
sdss_services = Registry.query(keyword='sdss', service_type='image')

print(f'{len(sdss_services)} result(s) found.')

sdss_services

21 result(s) found.


waveband,short_name,ivoid,res_description,access_url,reference_url,publisher,service_type
str8,str14,str38,str1285,str153,str49,str41,str17
infrared,SpIES,ivo://irsa.ipac/spitzer/images/spies,"The Spitzer IRAC Equatorial Survey (SpIES) is a large-area survey of 115 sq. degrees in the Equatorial SDSS Stripe 82 field. SpIES achieves 5 sigma depths of 6.13 microJy (21.93 AB magnitude) and 5.75 microJy (22.0 AB magnitude) at 3.6 and 4.5 microns, respectively.",https://irsa.ipac.caltech.edu/cgi-bin/Atlas/nph-atlas?mission=SpIES&amp;hdr_location=%5CSpIESDataPath%5C&amp;SIAP_ACTIVE=1&amp;collection_desc=SpIES&amp;,https://irsa.ipac.caltech.edu/data/SPITZER/SpIES/,NASA/IPAC Infrared Science Archive,simpleimageaccess
radio,VLA.AL218,ivo://mast.stsci/siap/al218,"The VLA-A Texas Survey consists of a sample of objects extracted from the earlier Texas Interferometer 365 MHz Survey of radio sources covering a strip of sky from approximately -35.5 degrees declination to +71.5 degrees declination, and complete to flux densities of 0.25 Jy, with positional accuracies of ~1 arcsecond in RA and DEC. The sample is a subset of 71 sources drawn from the area of one optical Schmidt sky survey plate (covering ~6.5x6.5 degrees), Region S861, centered at approximately RA=190.640822109, DEC=-0.273834224277 (J2000), from the UK Schmidt SRC-J Survey. The Region S861 was initially chosen because it represented the combination of the deepest UK Schmidt plate material (the best optical survey material available at the time of the sample definition in 1989) and the highest galactic latitude, thereby emphasizing the extragalactic nature of the survey and also maximizing the likelihood of having more optical detections. Much more recently, the area of this plate has been covered by a number of important sky surveys including 2MASS, NRAO VLA FIRST, and the Sloan Digital Sky Survey (SDSS) which is now public. In particular, the availability of the Sloan Survey data provides 5-band ugriz color information at optical wavelengths, to depth of g,r=22.2.",http://archive.stsci.edu/siap/search.php?id=al218&amp;,http://archive.stsci.edu/prepds/al218/,Space Telescope Science Institute Archive,simpleimageaccess
,ALHAMBRA_F814W,ivo://svo.cab/alhambra_siap_f814w,"This data server provides access to the F814W images of the ALHAMBRA Final Archive. The ALHAMBRA (Advance Large Homogeneous Area Medium Band Redshift Astronomical) survey (Moles et al. 2008) has observed 8 different regions of the sky, including sections of the COSMOS, DEEP2, ELAIS, GOODS-N, SDSS and Groth fields using a new photometric system with 20 contiguous, non-overlapping, equal width (~ 300A) filters, covering the optical range (3500A-9700A), plus the standard broadband NIR J, H and Ks filters. The observations were carried out with the Calar Alto (CAHA) 3.5m telescope using the wide field, 0.25 deg2 FOV optical camera LAICA and the NIR instrument Omega-2000. The ALHAMBRA survey dataset represents a ~700hrs of total exposure time, gathered in between the 2005 and 2012.Further information on the project can be found at the ALHAMBRA web page.",http://caha.sdc.cab.inta-csic.es/calto/siap_alhambra/alhambra_siap.jsp?,http://www.alhambrasurvey.com/,Spanish Virtual Observatory,simpleimageaccess
optical,SDSSDR2,ivo://sdss.jhu/services/siapdr2-images,"The Sloan Digital Sky Survey is a project to survey a 10000 square degree area on the Northern sky over a 5 year period. A dedicated 2.5m telescope is specially designed to take wide field (3 degrees in diameter) images using a 5x6 mosaic of 2048x2048 CCD`s, in five wavelength bands, operating in drift scan mode. The total raw data will exceed 40 TB. A processed subset, of about 1 TB in size, will consist of 1 million spectra, positions and image parameters for over 100 million objects, plus a mini-image centered on each object in every color. The data will be made available to the public after the completion of the survey",http://skyserver.sdss.org/vo/DR2SIAP/SIAP.asmx

# For each position, show the corresponding WISE, GALEX, and SDSS images.

In [ ]:
from astropy.coordinates import SkyCoord  # High-level coordinates
from navo_utils.image import Image
import astropy.units as u #is this needed?

    missions = ['GALEX', '2MASS', 'AllWISE']

    base_urls = ['http://mast.stsci.edu/portal_vo/Mashup/VoQuery.asmx/SiaV1?MISSION=GALEX&amp;', 
             'http://irsa.ipac.caltech.edu/ibe/sia/twomass/allsky/allsky?',
             'https://irsa.ipac.caltech.edu/ibe/sia/wise/allwise/p3am_cdd?']

    #Create an astropy table with this info.
    search_list = Table([missions, base_urls], names = ('mission', 'access_url'))

    size = '0'  #diameter in degrees

for object in galaxies:
    pos = SkyCoord(object['RA(deg)'], object['DEC(deg)'], frame="icrs", unit=(u.deg, u.deg))
    
    for mission in missions:
        
        wise_results = Image.query(coords=pos, radius=size, service=wise_services[0])
        table = wise_results[0]
        table.show_in_notebook()
        galex_results = Image.query(coords=pos, radius=size, service=galex_services[0])

        sdss_results = Image.query(coords=pos, radius=size, service=sdss_services[0])
        
        #xid = SDSS.query_region(pos, spectro=True) #requires matches to have spectroscopy, not just photometry:
        #im = SDSS.get_images(matches=xid, band='g')

## Grab the SDSS images for each super spiral

In [188]:
from astroquery.sdss import SDSS
from astropy.coordinates import SkyCoord  # High-level coordinates
import astropy.units as u

for object in galaxies:
    pos = SkyCoord(object['RA(deg)'], object['DEC(deg)'], frame="icrs", unit=(u.deg, u.deg))
    xid = SDSS.query_region(pos, spectro=True) #requires matches to have spectroscopy, not just photometry:
    im = SDSS.get_images(matches=xid, band='g')